In [ ]:
from torch import nn
from torch.nn.functional import relu
import torch
import os
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from lib.models import Gandalf
from lib.utils import *

from torch.utils.data import DataLoader
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

In [ ]:
FOLD = 1

folds = get_leave_one_out_cv_ids_for_ekyn()
train_ids,test_ids = folds[FOLD]
X,y = load_eeg_label_pair(id=train_ids[0],condition='PF')
testloader = DataLoader(Windowset(X,y),batch_size=32,shuffle=False)
print(f'testloader: {len(testloader)} batches')

model = Gandalf()
model.load_state_dict(torch.load(f=f'{GANDALFS_PATH}/gandalf_{FOLD}/best_model.pt',map_location='cpu',weights_only=False))
model.to(DEVICE)

criterion = torch.nn.CrossEntropyLoss()
loss,y_true,y_pred = evaluate(testloader,model,criterion,'cuda')
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
report = classification_report(y_true,y_pred,output_dict=True)

In [ ]:
report

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

folds = get_leave_one_out_cv_ids_for_ekyn()
criterion = torch.nn.CrossEntropyLoss()
reports = []

for FOLD,(train_ids,test_ids) in tqdm(enumerate(folds)):
    model = Gandalf()
    model.load_state_dict(torch.load(f=f'{GANDALFS_PATH}/gandalf_{FOLD}/best_model.pt',map_location='cpu',weights_only=False))
    model.to(DEVICE)
    for condition in ['Vehicle','PF']:
        X,y = load_eeg_label_pair(id=test_ids[0],condition=condition)
        testloader = DataLoader(Windowset(X,y),batch_size=32,shuffle=False)
        loss,y_true,y_pred = evaluate(testloader,model,criterion,'cuda')
        report = classification_report(y_true,y_pred,output_dict=True)
        reports.append(report)

In [ ]:
[report['macro avg']['f1-score'] for report in reports]